In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def preprocessing(fname):
    list_time = [] # time when fingers moved (str)
    with open(fname + '_time.txt') as f:
        for line in f:
            list_time.append(line.split('\t')[2].rstrip('\n'))
    list_seconds = [] # time when finger moved (float)
    for time in list_time:
        l = time.split(':')
        list_seconds.append(float(l[0]) * 3600 + float(l[1]) * 60 + float(l[2]))
    data = pd.read_csv(fname + '.txt', names=['time', 'ch1', 'ch2', 'ch3', 'ch4']) # wave data with time
    data['move'] = 0 # movement of finger
    for time in list_seconds: #match wave and the time fingers moved
        i = abs(data['time'] - time).idxmin()
        if fname[0] == 'R':
            data.at[data.index[i], 'move'] = 1
        else:
            data.at[data.index[i], 'move'] = -1
    data = data[(data['time'] > 30) & (data['time'] < data['time'].max() - 20)]
    data['inner'] = data['ch3'] - data['ch2']
    data['outer'] = data['ch4'] - data['ch1']
    data = data.drop(['ch1', 'ch2', 'ch3', 'ch4'], axis=1).reset_index(drop=True)
    n = len(data)
    for i in range(n - 1):
        if data.at[i + 1, 'time'] - data.at[i, 'time'] > 0.1:
            data = data.append((data.loc[i] + data.loc[i + 1]) / 2, ignore_index=True)
    data.loc[n:, 'move'] = 0
    data.sort_values(by='time', ascending=False)
    diff = pd.DataFrame(index=[], columns=['inner', 'outer'])
    for i in range(len(data) - 1):
        diff.loc[i] = data.loc[i + 1, ['inner', 'outer']] - data.loc[i, ['inner', 'outer']]
    return diff

In [ ]:
data = preprocessing('R1')

In [ ]:
data

In [ ]:
t = np.array(A['ch1'])[:n-6]
for i in range(1,7):
    t1 = np.array(A['ch1'])[i:n-6+i]
    t = np.vstack((t,t1))
    



In [ ]:
i = 0
t = np.array(A['ch1'])[i:(i-6)]

In [ ]:
t

In [ ]:
i = 1
t = np.vstack((t,np.array(A['ch1'])[i:(i-6)]))

In [ ]:
t

In [ ]:
np.array(A['ch1'])[:-6].shape

In [ ]:
t_0 = np.array(A['ch1'])[:-6]
t_1 = np.array(A['ch1'])[1:-5]

In [ ]:
l = len(t_0)

In [ ]:
B = pd.DataFrame(t_0, t_1)

In [ ]:
np.array(A['time'])[3:-3].shape

In [ ]:
B

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
t1 = np.array(R1['time'][:-1])
t2 = np.array(R1['time'][1:])
x = t1
y = t2 - t1
plt.scatter(x, y)
plt.show()